In [2]:
print("2주차 RAG 실습")


2주차 RAG 실습


In [ ]:
#FewShotPromtTemplate

from langchain_openai import ChatOpenAI                    
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate  
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler  # 스트리밍 출력 

# 1) LLM(챗GPT) 초기화 ─ 토큰이 생성될 때마다 바로바로 콘솔로 출력합니다.
chat = ChatOpenAI(
    model="gpt-4o-mini",  
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

# 2) ‘예시 Q&A’ 3개(모두 한글)
examples = [
    {
    "question": "프랑스에 대해 무엇을 알고 있나요?",
    "answer": """
    제가 알고 있는 것은 다음과 같습니다:
    - 수도: 파리
    - 언어: 프랑스어
    - 대표 음식: 와인과 치즈
    - 통화: 유로
""",
    },
    {
    "question": "이탈리아에 대해 무엇을 알고 있나요?",
    "answer": """
    제가 알고 있는 것은 다음과 같습니다:
    - 수도: 로마
    - 언어: 이탈리아어
    - 대표 음식: 피자와 파스타
    - 통화: 유로
""",
    },
    {
    "question": "그리스에 대해 무엇을 알고 있나요?",
    "answer": """
    제가 알고 있는 것은 다음과 같습니다:
    - 수도: 아테네
    - 언어: 그리스어
    - 대표 음식: 수블라키와 페타 치즈
    - 통화: 유로
""",
    },
]

# 3) 예시를 프롬프트에 끼워 넣는 서식 지정
example_prompt = PromptTemplate.from_template(
    "사람: {question}\n AI: {answer}"
)

# 4) Few-Shot 프롬프트 생성
prompt = FewShotPromptTemplate(
    examples=examples,                   # Q&A 예시
    example_prompt=example_prompt,       # 예시를 어떻게 표현할지
    suffix="사람: {country}에 대해 무엇을 알고 있나요?\nAI:",  # 실제 사용자 질문 자리
    input_variables=["country"],         # 외부에서 넣을 변수 이름
)

# 5) LCEL(≒ 파이프라인)로 프롬프트 → LLM 연결
chain = prompt | chat

# 6) 체인 실행: ‘터키’ 정보를 요청
chain.invoke({"country": "한국"})


제가 알고 있는 것은 다음과 같습니다:
- 수도: 서울
- 언어: 한국어
- 대표 음식: 김치, 비빔밥, 불고기
- 통화: 원 (KRW)

AIMessage(content='제가 알고 있는 것은 다음과 같습니다:\n- 수도: 서울\n- 언어: 한국어\n- 대표 음식: 김치, 비빔밥, 불고기\n- 통화: 원 (KRW)', response_metadata={'finish_reason': 'stop'}, id='run-8bfb4f27-4ed9-4505-898b-c43696070e6f-0')

In [8]:
#FewShotChatMessagePromptTemplate

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 1) LLM 준비
chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

# 2) 예시 Q&A (한글)
examples = [
    {
        "country": "프랑스",
        "answer": """
제가 알고 있는 것은 다음과 같습니다:
- 수도: 파리
- 언어: 프랑스어
- 대표 음식: 와인과 치즈
- 통화: 유로
""",
    },
    {
        "country": "이탈리아",
        "answer": """
제가 알고 있는 것은 다음과 같습니다:
- 수도: 로마
- 언어: 이탈리아어
- 대표 음식: 피자와 파스타
- 통화: 유로
""",
    },
    {
        "country": "그리스",
        "answer": """
제가 알고 있는 것은 다음과 같습니다:
- 수도: 아테네
- 언어: 그리스어
- 대표 음식: 수블라키와 페타 치즈
- 통화: 유로
""",
    },
]

# 3) 예시를 묶는 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{country}에 대해 무엇을 알고 있나요?"),
     ("ai", "{answer}")]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 4) 최종 프롬프트 (시스템+예시+실제 질문)
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 지리 전문가이며 짧게 대답합니다."),
        few_shot_prompt,
        ("human", "{country}에 대해 무엇을 알고 있나요?"),
    ]
)

# 5) 체인 만들기
chain = final_prompt | chat

# 6) 실행 : ‘한국’ 정보 요청
chain.invoke({"country": "한국"})



제가 알고 있는 것은 다음과 같습니다:
- 수도: 서울
- 언어: 한국어
- 대표 음식: 김치와 비빔밥
- 통화: 원 (KRW)

AIMessage(content='\n제가 알고 있는 것은 다음과 같습니다:\n- 수도: 서울\n- 언어: 한국어\n- 대표 음식: 김치와 비빔밥\n- 통화: 원 (KRW)', response_metadata={'finish_reason': 'stop'}, id='run-e56dbef6-c9b7-4575-b224-677b3e34ee76-0')

In [1]:
# 롤플레잉 어시스턴트 파이프라인 프롬프트 예제

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, PipelinePromptTemplate
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 1) GPT 모델 준비
chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

# 2) 파이프라인용 작은 템플릿 세 개
intro = PromptTemplate.from_template(
    """
당신은 역할극 어시스턴트입니다.
지금부터 {character}(을)를 흉내 냅니다.
"""
)

example = PromptTemplate.from_template(
    """
이것은 당신의 말투 예시입니다:

사람: {example_question}
당신: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
자, 시작합니다!

사람: {question}
당신:
"""
)

# 3) 세 조각을 하나로 합칠 큰 템플릿
final = PromptTemplate.from_template(
    """
{intro}

{example}

{start}
"""
)

# 4) 파이프라인 프롬프트 설정
prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

# 5) 체인 만들고 실행
chain = full_prompt | chat

chain.invoke(
    {
        "character": "늑대",
        "example_question": "당신의 위치가 어디인가요?",
        "example_answer": "으르렁! 그건 비밀이다! 으르렁!",
        "question": "가장 좋아하는 음식은 무엇인가요?",
    }
)


으르렁! 나는 고기와 신선한 생선이 제일 좋아! 으르렁!

AIMessage(content='으르렁! 나는 고기와 신선한 생선이 제일 좋아! 으르렁!', response_metadata={'finish_reason': 'stop'}, id='run-28e05e7a-cabf-4977-a38a-54be09e9574f-0')

In [ ]:
# LLM 캐싱(SQLite) 사용 예제
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache   # ← 경로를 langchain.cache 로 수정!

# 1) GPT 결과를 디스크(SQLite)에 저장하도록 캐시 설정
set_llm_cache(SQLiteCache("cache.db"))

# 2) GPT 모델 준비
chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

# 3) 질문 보내고 답 받기
print(chat.invoke("이탈리아 파스타는 어떻게 만들어요?"))


content='이탈리아 파스타는 다양한 종류와 조리법이 있지만, 기본적인 파스타 반죽을 만드는 방법과 간단한 소스를 소개해드릴게요.\n\n### 기본 파스타 반죽 만들기\n\n#### 재료:\n- 밀가루 (00형 또는 일반 밀가루) 100g\n- 계란 1개\n- 소금 약간\n\n#### 방법:\n1. **밀가루 준비**: 깨끗한 작업대에 밀가루를 쌓고 가운데에 우물을 만듭니다.\n2. **계란 넣기**: 우물 중앙에 계란을 깨고 소금을 추가합니다.\n3. **반죽하기**: 포크를 사용해 계란을 휘저어가며 밀가루를 조금씩 섞어줍니다. 반죽이 뭉쳐지기 시작하면 손으로 반죽을 치대어 줍니다.\n4. **반죽 숙성**: 반죽이 부드럽고 탄력있게 될 때까지 약 10분간 치대고, 랩으로 싸서 30분 정도 숙성시킵니다.\n\n### 파스타 모양 만들기\n1. 숙성된 반죽을 밀대로 얇게 밀어줍니다.\n2. 원하는 모양(스파게티, 페투치니 등)으로 잘라줍니다.\n3. 잘라낸 파스타는 밀가루를 뿌려 서로 붙지 않도록 합니다.\n\n### 간단한 토마토 소스 만들기\n\n#### 재료:\n- 올리브 오일 2큰술\n- 다진 양파 1개\n- 다진 마늘 2쪽\n- 토마토 (신선한 것 또는 통조림) 400g\n- 소금, 후추, 바질 (선택사항)\n\n#### 방법:\n1. **양파와 마늘 볶기**: 팬에 올리브 오일을 두르고 중불에서 양파와 마늘을 볶아 향이 나게 합니다.\n2. **토마토 추가**: 다진 토마토를 넣고 소금과 후추로 간을 합니다. 약 15-20분간 끓여 소스가 걸쭉해지도록 합니다.\n3. **바질 추가**: 마지막에 바질을 넣어 향을 더합니다.\n\n### 파스타 조리\n1. 끓는 물에 소금을 넣고, 준비한 파스타를 넣어 2-4분 정도 삶습니다 (신선한 파스타는 짧은 시간에 익습니다).\n2. 삶은 파스타를 체에 걸러 물기를 제거하고, 소스와 섞어줍니다.\n\n이제 맛있는 이탈리아 파스타가 완성되었습니다! 추가로 치즈나 다른 재료를 넣어 취향에 맞게 즐기세요.' re

In [9]:
# 대화 메모리 챗봇 ─ 최종 안정 버전
# pip install -U langchain-openai langchain

from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

# 1) GPT 모델
bot = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# 2) 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 챗봇입니다."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),            # ← input 키 사용
    ]
)

# 3) 메모리 (키 이름 지정)
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="history",  # 프롬프트에 쓸 이름
    input_key="input",     # 사용자가 준 질문이 담길 키
    output_key="output",   # GPT 답이 담길 키
)

# 4) 메모리 → 프롬프트로 보내는 함수 (리스트만 반환)
def load_history(_):
    return memory.load_memory_variables({})["history"]

# 5) 파이프라인
chain = (
    RunnablePassthrough.assign(history=load_history)
    | prompt
    | bot
)

# ── 실제 대화 테스트 ──────────────────
# ▶ 첫 인사
user_1 = {"input": "안녕? 나는 현빈이야!"}
ai_1 = chain.invoke(user_1)
print("GPT 1차:", ai_1.content)

# 메모리에 문자열만 저장
memory.save_context(user_1, {"output": ai_1.content})

# ▶ 두 번째 질문
user_2 = {"input": "내 이름 기억해?"}
ai_2 = chain.invoke(user_2)
print("GPT 2차:", ai_2.content)

memory.save_context(user_2, {"output": ai_2.content})


GPT 1차: 안녕하세요, 현빈님! 만나서 반가워요. 어떻게 도와드릴까요?
GPT 2차: 네, 현빈님이라고 말씀하셨죠! 기억하고 있어요. 다른 질문이나 이야기하고 싶은 것이 있으면 언제든지 말씀해 주세요!


In [ ]:
# 🪟 Window-Memory 챗봇 (k=4) 간단 데모
# pip install -U langchain-openai langchain

from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

# 1) GPT 모델
bot = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# 2) 최근 4개 메시지만 보관하는 메모리
memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,                       # 창 크기
    memory_key="history",
    input_key="input",
    output_key="output",
)

# 3) 프롬프트 (= 시스템 지시 + 과거 기록 + 새 질문)
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 키움증권 AI 상담원입니다."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

# 4) 메모리를 프롬프트에 주입하는 함수
def load_history(_):
    return memory.load_memory_variables({})["history"]

# 5) 파이프라인
chat_chain = RunnablePassthrough.assign(history=load_history) | prompt | bot

# 6) 7개 메시지로 테스트
script = [
    "안녕하세요! 제 이름은 홍길동입니다.",            # (1)
    "제 계좌 잔액은 300만 원이에요.",                # (2) *사라질 정보
    "오늘 코스피 마감 지수 알려주세요.",              # (3)
    "삼성전자 현재 주가도 궁금합니다.",               # (4)
    "이번 달 원‧달러 환율 최고치는요?",              # (5)
    "제 포트폴리오 1개월 수익률이 궁금해요.",         # (6)
    "제가 처음에 말한 계좌 잔액이 얼마였죠?",         # (7) ← 기억 여부 확인
]

for idx, line in enumerate(script, 1):
    human = {"input": line}
    ai = chat_chain.invoke(human)                       # GPT 응답
    memory.save_context(human, {"output": ai.content})  # 메모리에 문자열만 저장

    print(f"\n🗣️  [사용자 {idx}] {line}")
    print(f"🤖 [AI {idx}] {ai.content}")

    # 현재 남아 있는 4개 메시지 보여 주기
    print("📚 현재 메모리:")
    for msg in memory.load_memory_variables({})["history"]:
        role = "사용자" if msg.type == "human" else "AI "
        print(f"   - ({role}) {msg.content}")
    print("-" * 55)



🗣️  [사용자 1] 안녕하세요! 제 이름은 홍길동입니다.
🤖 [AI 1] 안녕하세요, 홍길동님! 키움증권 AI 상담원입니다. 어떻게 도와드릴까요?
📚 현재 메모리:
   - (사용자) 안녕하세요! 제 이름은 홍길동입니다.
   - (AI ) 안녕하세요, 홍길동님! 키움증권 AI 상담원입니다. 어떻게 도와드릴까요?
-------------------------------------------------------

🗣️  [사용자 2] 제 계좌 잔액은 300만 원이에요.
🤖 [AI 2] 홍길동님, 계좌 잔액이 300만 원이군요. 어떤 정보를 원하시거나 도움이 필요하신 부분이 있으신가요? 예를 들어, 투자 상품, 주식 거래, 또는 다른 금융 서비스에 대해 궁금한 점이 있으시면 말씀해 주세요!
📚 현재 메모리:
   - (사용자) 안녕하세요! 제 이름은 홍길동입니다.
   - (AI ) 안녕하세요, 홍길동님! 키움증권 AI 상담원입니다. 어떻게 도와드릴까요?
   - (사용자) 제 계좌 잔액은 300만 원이에요.
   - (AI ) 홍길동님, 계좌 잔액이 300만 원이군요. 어떤 정보를 원하시거나 도움이 필요하신 부분이 있으신가요? 예를 들어, 투자 상품, 주식 거래, 또는 다른 금융 서비스에 대해 궁금한 점이 있으시면 말씀해 주세요!
-------------------------------------------------------

🗣️  [사용자 3] 오늘 코스피 마감 지수 알려주세요.
🤖 [AI 3] 죄송하지만, 실시간 데이터나 특정 날짜의 마감 지수를 제공할 수는 없습니다. 하지만 코스피 지수는 주식 시장의 주요 지표 중 하나로, 한국거래소(KRX) 웹사이트나 금융 뉴스 사이트에서 확인하실 수 있습니다. 다른 궁금한 점이나 도움이 필요하신 부분이 있으시면 말씀해 주세요!
📚 현재 메모리:
   - (사용자) 안녕하세요! 제 이름은 홍길동입니다.
   - (AI ) 안녕하세요, 홍길동님! 키움증권 AI 상담원입니다. 어떻게 도와드릴까요?
   - (사용자

In [34]:
# 요약 메모리 챗봇 + 요약 한국어 번역
# pip install -U langchain-openai langchain

from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryMemory

# 1) GPT-4o-mini 모델
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# 2) 요약 메모리
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="summary",
    input_key="input",
    output_key="output",
)

# 3) 대화 저장 함수
def add_message(user, ai):
    memory.save_context({"input": user}, {"output": ai})

# 4) 예시 대화 3개
add_message("안녕하세요! 저는 현빈입니다. 한국에 살아요.", "와! 정말 멋지네요!")
add_message("한국은 정말 아름다워요.", "저도 꼭 가보고 싶어요!")
add_message("특히 제주도가 최고예요.", "제주도 이야기를 더 들려주세요!")

# 5) 요약 가져오기
summary = memory.load_memory_variables({})["summary"]
print(summary)


The human introduces themselves as Hyun-bin and mentions they live in Korea, to which the AI responds positively, expressing that it's really cool. The human then comments that Korea is very beautiful, particularly highlighting Jeju Island as the best. The AI expresses a desire to hear more about Jeju Island.


In [35]:
translate_summary = llm.invoke(f"한국어로 요약해주세요: {summary}")
print(translate_summary.content)

인간은 자신을 현빈이라고 소개하며 한국에 살고 있다고 말하고, AI는 긍정적으로 반응하며 정말 멋지다고 표현합니다. 인간은 한국이 매우 아름답다고 언급하며 특히 제주도를 최고의 장소로 강조합니다. AI는 제주도에 대해 더 듣고 싶다고 말합니다.


In [36]:
# 1. 상자(라이브러리) 불러오기
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
import tiktoken                # 글자 조각(토큰) 세어 주는 도구
from types import MethodType   # 함수 붙여 주는 도구

# 2. GPT 가 글자 조각을 헤아릴 수 있게 새 함수 달아 주기
def count_tokens(self, messages):
    enc = tiktoken.get_encoding("cl100k_base")  # 토큰 규칙
    total = 0
    for msg in messages:
        total += 4                    # 역할·구분 기호 4토큰
        total += len(enc.encode(msg.content))  # 내용 토큰
    total += 2                        # 대화 끝 표시
    return total

# 3. ChatOpenAI 클래스에 함수 붙이기
ChatOpenAI.get_num_tokens_from_messages = MethodType(count_tokens, ChatOpenAI)

print("✅ 준비 완료! 이제 토큰 숫자를 잘 셀 수 있어요.")


✅ 준비 완료! 이제 토큰 숫자를 잘 셀 수 있어요.


In [37]:
bot = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# 요약 버퍼” 노트 만들기 (80토큰 넘으면 압축!)
memory = ConversationSummaryBufferMemory(
    llm=bot,
    max_token_limit=80,   # 80토큰 이상이면 앞 내용을 요약
    return_messages=True  # 메시지 모양 그대로 보여 줘
)

# 3. 대화 저장하는 간단 함수
def speak(user, ai):
    memory.save_context({"input": user}, {"output": ai})

# 4. 노트 안을 보여 주는 함수
def show(title):
    print(f"\n📝 {title}")
    for m in memory.load_memory_variables({})["history"]:
        who = "USER" if m.type == "human" else "AI  "
        print(f"[{who}] {m.content}")

# ── 실제 대화 넣어 보기 ─────────────────
speak("안녕하세요! 저는 현빈입니다. 한국에 살아요.", "우와, 정말 멋지네요!")
speak("한국은 사계절이 뚜렷해서 아름다워요.", "저도 꼭 가보고 싶어요!")
show("① 대화 두 개 (아직 요약 X)")

# 토큰을 좀 더 늘려 요약이 생기도록!
speak("특히 봄 벚꽃 축제가 최고예요.", "벚꽃 사진이 있다면 보고 싶네요!")
speak("김치와 불고기도 정말 맛있죠!", "불고기라니, 벌써 군침이 도네요!")
show("② 대화 네 개 (80토큰 ↑ → 앞부분이 영문 요약으로 변신!)")



📝 ① 대화 두 개 (아직 요약 X)
[AI  ] The human introduces themselves as Hyun-bin and mentions that they live in Korea.
[AI  ] 우와, 정말 멋지네요!
[USER] 한국은 사계절이 뚜렷해서 아름다워요.
[AI  ] 저도 꼭 가보고 싶어요!

📝 ② 대화 네 개 (80토큰 ↑ → 앞부분이 영문 요약으로 변신!)
[AI  ] The human introduces themselves as Hyun-bin and mentions that they live in Korea. The AI expresses admiration, and the human adds that Korea is beautiful because it has distinct four seasons. The AI expresses a desire to visit Korea, and the human highlights that the cherry blossom festival in spring is the best.
[AI  ] 벚꽃 사진이 있다면 보고 싶네요!
[USER] 김치와 불고기도 정말 맛있죠!
[AI  ] 불고기라니, 벌써 군침이 도네요!


In [43]:
# 지식 그래프 메모리 예제

from langchain_openai import ChatOpenAI
from langchain.memory import ConversationKGMemory

# 1) LLM 준비
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# 2) 지식 그래프 메모리
memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,  # 메시지까지 보려면 True
)

# 3) 대화 두 번 저장
memory.save_context(
    {"input": "안녕! 나는 현빈이고, 한국에 살고 있어."},
    {"output": "우와, 정말 멋지네요!"}
)
memory.save_context(
    {"input": "현빈이는 김치를 좋아해."},
    {"output": "우와, 그거 정말 맛있죠!"}
)


In [44]:
print("\n[질문] 현빈은 누구?")
print(memory.load_memory_variables({"input": "현빈은 누구야?"})["history"])

print("\n[질문] 현빈이가 좋아하는 것은?")
print(memory.load_memory_variables({"input": "현빈이는 뭘 좋아해?"})["history"])


[질문] 현빈은 누구?
[SystemMessage(content='On 현빈: 현빈 lives in 한국. 현빈 좋아해 김치.')]

[질문] 현빈이가 좋아하는 것은?
[SystemMessage(content='On 현빈: 현빈 lives in 한국. 현빈 좋아해 김치.')]


In [48]:
# ConversationSummaryBufferMemory 실습
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import tiktoken, types

# ── 1. ChatOpenAI에 ‘토큰 세기’ 기능 패치 ──
def count_tokens(self, messages):
    enc = tiktoken.get_encoding("cl100k_base")
    total = 0
    for msg in messages:
        total += 4                           # role 관련 고정 토큰
        total += len(enc.encode(msg.content))
    total += 2                               # 대화 종료 토큰
    return total

ChatOpenAI.get_num_tokens_from_messages = types.MethodType(count_tokens, ChatOpenAI)

# ── 2. GPT 모델 & 요약 버퍼 메모리 ──
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,      # 120토큰 초과 시 앞부분 자동 요약
    memory_key="chat_history",
    return_messages=True      # 메시지 객체 그대로 유지!
)

# ── 3. 프롬프트 & 체인 ──
prompt = PromptTemplate.from_template("""
You are a helpful AI talking to a human.

{chat_history}
Human: {question}
You:""".strip())

chain = LLMChain(llm=llm, memory=memory, prompt=prompt, verbose=False)

# ── 4. 대화 3번 ──
print("▶ 1) ", chain.predict(question="내이름은 현빈입니다."))
print("▶ 2) ", chain.predict(question="저는 한국에 살고있어요"))
print("▶ 3) ", chain.predict(question="너의 이름은 무엇이야?"))

# ── 5. 메모리 내용 출력 ──
print("\n📚 메모리에 저장된 chat_history:")
hist = memory.load_memory_variables({})["chat_history"]
for msg in hist:
    if hasattr(msg, "type"):                       # 메시지 객체일 때
        role = "USER" if msg.type == "human" else "AI  "
        print(f"[{role}] {msg.content}")
    else:                                          # 혹시 요약 문자열이면 그대로
        print(msg)


▶ 1)  안녕하세요, 현빈님! 만나서 반갑습니다. 어떻게 도와드릴까요?
▶ 2)  그렇군요! 한국은 정말 아름다운 나라죠. 어떤 지역에 살고 계신가요?
▶ 3)  저는 AI이기 때문에 이름이 없어요. 하지만 필요하신 정보나 도움이 필요하시면 언제든지 말씀해 주세요!

📚 메모리에 저장된 chat_history:
[AI  ] The human introduces himself as Hyun-bin and mentions that he lives in Korea. The AI greets him in Korean and asks how it can assist him.
[AI  ] 그렇군요! 한국은 정말 아름다운 나라죠. 어떤 지역에 살고 계신가요?
[USER] 너의 이름은 무엇이야?
[AI  ] 저는 AI이기 때문에 이름이 없어요. 하지만 필요하신 정보나 도움이 필요하시면 언제든지 말씀해 주세요!
